In [1]:
# 이거 먼저 설치하고 런타임 재시작 할 것
# 이거 설치 안하면 plt 한국어가 깨져서 보임
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 7 not upgraded.
Need to get 9,604 kB of archives.
After this operation, 29.5 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fonts-nanum all 20170925-1 [9,604 kB]
Fetched 9,604 kB in 1s (9,004 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype

# load the dataset

In [2]:
import pandas as pd
import numpy as np

np.random.seed(42)

from matplotlib import pyplot
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns

mpl.rcParams['axes.unicode_minus'] = False
mpl.rc("font", family = "Malgun Gothic")

from sklearn.metrics import silhouette_score

In [3]:
# 경고 없애는 코드
import warnings
warnings.filterwarnings(action='ignore')

plt.rc('font', family='NanumBarunGothic') 

In [4]:
df = pd.read_excel('/content/preprocessing_data.xlsx')
df

,서울시 동,가구원수,서울시 구,구별 가구수,반려동물 비율,반려동물 가구수 구,반려동물 가구수 동,면적,반려동물 가구밀도,폐사안락사수,보호소 수,보호소 밀도,폐사안락사 비율,동물병원약국개수,동물병원약국 밀도
0,가락1동,9534,송파구,255766,0.216,55245.456,2059.344,1.34,1536.823881,15,1,0.746269,11.194030,11,8.208955
1,가락2동,11482,송파구,255766,0.216,55245.456,2480.112,0.96,2583.450000,13,1,1.041667,13.541667,11,11.458333
2,가락본동,10576,송파구,255766,0.216,55245.456,2284.416,1.13,2021.607080,19,1,0.884956,16.814159,10,8.849558
3,가리봉동,4901,구로구,164083,0.195,31996.185,955.695,0.40,2389.237500,0,0,0.000000,0.000000,2,5.000000
4,가산동,14124,금천구,103432,0.179,18514.328,2528.196,2.52,1003.252381,0,0,0.000000,0.000000,14,5.555556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,청구동,5764,중구,55093,0.191,10522.763,1100.924,0.34,3238.011765,0,0,0.000000,0.000000,2,4.545455
421,동화동,3771,중구,55093,0.191,10522.763,720.261,0.26,2770.234615,0,0,0.000000,0.000000,2,2.325581
422,항동,5988,구로구,164083,0.195,31996.185,1167.660,1.40,834.042857,0,0,0.000000,0.000000,2,3.174603
423,위례동,9041,송파구,255766,0.216,55245.456,1952.856,2.55,765.825882,0,0,0.000000,0.000000,2,1.904762


In [5]:
# 독립변수 추출
col = ['서울시 동', '가구원수', '서울시 구', '구별 가구수', '반려동물 비율', '반려동물 가구수 구',
       '반려동물 가구수 동', '면적', '폐사안락사수', '보호소 수', '동물병원약국개수']
x = df.drop(col,axis=1)
x

,반려동물 가구밀도,보호소 밀도,폐사안락사 비율,동물병원약국 밀도
0,1536.823881,0.746269,11.194030,8.208955
1,2583.450000,1.041667,13.541667,11.458333
2,2021.607080,0.884956,16.814159,8.849558
3,2389.237500,0.000000,0.000000,5.000000
4,1003.252381,0.000000,0.000000,5.555556
...,...,...,...,...
420,3238.011765,0.000000,0.000000,4.545455
421,2770.234615,0.000000,0.000000,2.325581
422,834.042857,0.000000,0.000000,3.174603
423,765.825882,0.000000,0.000000,1.904762


# Scaler

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaled_x = scaler.fit_transform(x) # 설명변수

In [7]:
df_scaled_x = pd.DataFrame(scaled_x,columns = x.columns)
df_scaled_x

,반려동물 가구밀도,보호소 밀도,폐사안락사 비율,동물병원약국 밀도
0,-0.378614,0.160694,0.129900,0.243147
1,0.711349,0.543612,0.292286,0.926818
2,0.126243,0.340471,0.518644,0.377930
3,0.509095,-0.806678,-0.644390,-0.432020
4,-0.934279,-0.806678,-0.644390,-0.315130
...,...,...,...,...
420,1.393014,-0.806678,-0.644390,-0.527656
421,0.905868,-0.806678,-0.644390,-0.994719
422,-1.110495,-0.806678,-0.644390,-0.816084
423,-1.181536,-0.806678,-0.644390,-1.083260


# Making K means clustering k = 93

In [8]:
from sklearn.cluster import KMeans

np.random.seed(42)

n_clusters=93

kmeans = KMeans(n_clusters).fit(scaled_x)

In [9]:
kmeans.cluster_centers_
# 반려동물 가구밀도     #보호소 밀도    #폐사안락사 비율(폐사안락사 수/면적)    #동물병원약국 밀도

array([[-1.45011872e-01, -5.97601224e-01, -5.21669521e-01,
         2.51063689e+00],
       [-4.38225728e-01,  2.50629500e-01,  4.02869536e-01,
        -3.95244754e-01],
       [ 1.14167367e+00, -8.06678444e-01, -6.44390355e-01,
        -5.45663967e-01],
       [ 1.08442959e+00,  1.42828494e+00,  1.97929645e+00,
         1.16851632e-01],
       [-2.10298825e-01, -8.06678444e-01, -6.44390355e-01,
        -7.50739949e-01],
       [ 1.55691475e+00,  1.07198643e+00,  4.86915437e+00,
         1.34225472e+00],
       [-1.26052636e+00, -7.44057465e-01, -5.92878426e-01,
         2.82793480e-01],
       [ 1.28964689e+00,  1.24350948e+00, -2.12955254e-01,
        -5.35079065e-01],
       [ 5.21302753e-02,  8.59675782e-01,  8.78762330e-01,
         1.33035388e+00],
       [ 6.92082360e-01,  2.70704420e+00,  1.69045299e-02,
         4.45226559e-01],
       [ 1.56651518e+00, -8.06678444e-01, -6.44390355e-01,
         1.38008342e+00],
       [-1.12500185e+00, -2.20356295e-01, -3.24944754e-01,
      

In [10]:
df_train = df.copy()
df_train["target"] = kmeans.labels_
df_train

,서울시 동,가구원수,서울시 구,구별 가구수,반려동물 비율,반려동물 가구수 구,반려동물 가구수 동,면적,반려동물 가구밀도,폐사안락사수,보호소 수,보호소 밀도,폐사안락사 비율,동물병원약국개수,동물병원약국 밀도,target
0,가락1동,9534,송파구,255766,0.216,55245.456,2059.344,1.34,1536.823881,15,1,0.746269,11.194030,11,8.208955,36
1,가락2동,11482,송파구,255766,0.216,55245.456,2480.112,0.96,2583.450000,13,1,1.041667,13.541667,11,11.458333,30
2,가락본동,10576,송파구,255766,0.216,55245.456,2284.416,1.13,2021.607080,19,1,0.884956,16.814159,10,8.849558,39
3,가리봉동,4901,구로구,164083,0.195,31996.185,955.695,0.40,2389.237500,0,0,0.000000,0.000000,2,5.000000,40
4,가산동,14124,금천구,103432,0.179,18514.328,2528.196,2.52,1003.252381,0,0,0.000000,0.000000,14,5.555556,82
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,청구동,5764,중구,55093,0.191,10522.763,1100.924,0.34,3238.011765,0,0,0.000000,0.000000,2,4.545455,2
421,동화동,3771,중구,55093,0.191,10522.763,720.261,0.26,2770.234615,0,0,0.000000,0.000000,2,2.325581,60
422,항동,5988,구로구,164083,0.195,31996.185,1167.660,1.40,834.042857,0,0,0.000000,0.000000,2,3.174603,82
423,위례동,9041,송파구,255766,0.216,55245.456,1952.856,2.55,765.825882,0,0,0.000000,0.000000,2,1.904762,70


In [11]:
df_scaled_x["cluster"] = kmeans.labels_
df_scaled_x

,반려동물 가구밀도,보호소 밀도,폐사안락사 비율,동물병원약국 밀도,cluster
0,-0.378614,0.160694,0.129900,0.243147,36
1,0.711349,0.543612,0.292286,0.926818,30
2,0.126243,0.340471,0.518644,0.377930,39
3,0.509095,-0.806678,-0.644390,-0.432020,40
4,-0.934279,-0.806678,-0.644390,-0.315130,82
...,...,...,...,...,...
420,1.393014,-0.806678,-0.644390,-0.527656,2
421,0.905868,-0.806678,-0.644390,-0.994719,60
422,-1.110495,-0.806678,-0.644390,-0.816084,82
423,-1.181536,-0.806678,-0.644390,-1.083260,70


# Select clusters which are larger than zero

In [12]:
df_cluster_mean = df_scaled_x.groupby('cluster').mean()
pet_over = df_cluster_mean['반려동물 가구밀도']>0
shelter_over = df_cluster_mean['보호소 밀도']>0
death_over = df_cluster_mean['폐사안락사 비율']>0
hospital_over = df_cluster_mean['동물병원약국 밀도']>0
df_over = df_cluster_mean[pet_over & shelter_over &death_over & hospital_over]

In [13]:
df_over

,반려동물 가구밀도,보호소 밀도,폐사안락사 비율,동물병원약국 밀도
cluster,,,,
3,1.084430,1.428285,1.979296,0.116852
5,1.556915,1.071986,4.869154,1.342255
8,0.052130,0.859676,0.878762,1.330354
9,0.692082,2.707044,0.016905,0.445227
13,0.610970,1.428285,0.071161,2.341444
21,1.953354,1.550192,2.499697,3.775994
29,0.739392,0.937142,0.742630,2.984428
37,1.382042,1.026459,1.145724,1.758722
39,0.193214,0.599826,0.035672,0.066951


In [14]:
df_over.index

Int64Index([3, 5, 8, 9, 13, 21, 29, 37, 39, 42, 66, 67, 68, 74, 83, 90], dtype='int64', name='cluster')

In [15]:
new_target = []

for i,t in enumerate(df_train['target']):
    if t not in df_over.index:
        new_target.append(0)
    else:
        new_target.append(1)

df_train['target'] = new_target
df_train

,서울시 동,가구원수,서울시 구,구별 가구수,반려동물 비율,반려동물 가구수 구,반려동물 가구수 동,면적,반려동물 가구밀도,폐사안락사수,보호소 수,보호소 밀도,폐사안락사 비율,동물병원약국개수,동물병원약국 밀도,target
0,가락1동,9534,송파구,255766,0.216,55245.456,2059.344,1.34,1536.823881,15,1,0.746269,11.194030,11,8.208955,0
1,가락2동,11482,송파구,255766,0.216,55245.456,2480.112,0.96,2583.450000,13,1,1.041667,13.541667,11,11.458333,0
2,가락본동,10576,송파구,255766,0.216,55245.456,2284.416,1.13,2021.607080,19,1,0.884956,16.814159,10,8.849558,1
3,가리봉동,4901,구로구,164083,0.195,31996.185,955.695,0.40,2389.237500,0,0,0.000000,0.000000,2,5.000000,0
4,가산동,14124,금천구,103432,0.179,18514.328,2528.196,2.52,1003.252381,0,0,0.000000,0.000000,14,5.555556,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,청구동,5764,중구,55093,0.191,10522.763,1100.924,0.34,3238.011765,0,0,0.000000,0.000000,2,4.545455,0
421,동화동,3771,중구,55093,0.191,10522.763,720.261,0.26,2770.234615,0,0,0.000000,0.000000,2,2.325581,0
422,항동,5988,구로구,164083,0.195,31996.185,1167.660,1.40,834.042857,0,0,0.000000,0.000000,2,3.174603,0
423,위례동,9041,송파구,255766,0.216,55245.456,1952.856,2.55,765.825882,0,0,0.000000,0.000000,2,1.904762,0


In [16]:
df_train.columns

Index(['서울시 동', '가구원수', '서울시 구', '구별 가구수', '반려동물 비율', '반려동물 가구수 구',
       '반려동물 가구수 동', '면적', '반려동물 가구밀도', '폐사안락사수', '보호소 수', '보호소 밀도',
       '폐사안락사 비율', '동물병원약국개수', '동물병원약국 밀도', 'target'],
      dtype='object')

In [17]:
x = df_train[['가구원수', '면적', '반려동물 가구수 동', '반려동물 가구밀도', '폐사안락사수', '폐사안락사 비율',
       '보호소 수', '보호소 밀도', '동물병원약국개수', '동물병원약국 밀도','target']]
x

,가구원수,면적,반려동물 가구수 동,반려동물 가구밀도,폐사안락사수,폐사안락사 비율,보호소 수,보호소 밀도,동물병원약국개수,동물병원약국 밀도,target
0,9534,1.34,2059.344,1536.823881,15,11.194030,1,0.746269,11,8.208955,0
1,11482,0.96,2480.112,2583.450000,13,13.541667,1,1.041667,11,11.458333,0
2,10576,1.13,2284.416,2021.607080,19,16.814159,1,0.884956,10,8.849558,1
3,4901,0.40,955.695,2389.237500,0,0.000000,0,0.000000,2,5.000000,0
4,14124,2.52,2528.196,1003.252381,0,0.000000,0,0.000000,14,5.555556,0
...,...,...,...,...,...,...,...,...,...,...,...
420,5764,0.34,1100.924,3238.011765,0,0.000000,0,0.000000,2,4.545455,0
421,3771,0.26,720.261,2770.234615,0,0.000000,0,0.000000,2,2.325581,0
422,5988,1.40,1167.660,834.042857,0,0.000000,0,0.000000,2,3.174603,0
423,9041,2.55,1952.856,765.825882,0,0.000000,0,0.000000,2,1.904762,0


# VIF

Remove if value is greater than 10


In [20]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

 
def feature_engineering_XbyVIF(X_train):
    vif = pd.DataFrame()
    vif['VIF_Factor'] = [variance_inflation_factor(X_train.values, i)
                         for i in range(X_train.shape[1])]
    vif['Feature'] = X_train.columns
    return vif

xx = x.drop(['가구원수','반려동물 가구수 동','폐사안락사수','보호소 수','target'],axis=1)

vif = feature_engineering_XbyVIF(xx)
print(vif)



   VIF_Factor    Feature
0    1.572632         면적
1    3.202478  반려동물 가구밀도
2    2.417843   폐사안락사 비율
3    2.983575     보호소 밀도
4    4.761741   동물병원약국개수
5    4.368103  동물병원약국 밀도


In [21]:
xx['target'] = x['target']
xx

,면적,반려동물 가구밀도,폐사안락사 비율,보호소 밀도,동물병원약국개수,동물병원약국 밀도,target
0,1.34,1536.823881,11.194030,0.746269,11,8.208955,0
1,0.96,2583.450000,13.541667,1.041667,11,11.458333,0
2,1.13,2021.607080,16.814159,0.884956,10,8.849558,1
3,0.40,2389.237500,0.000000,0.000000,2,5.000000,0
4,2.52,1003.252381,0.000000,0.000000,14,5.555556,0
...,...,...,...,...,...,...,...
420,0.34,3238.011765,0.000000,0.000000,2,4.545455,0
421,0.26,2770.234615,0.000000,0.000000,2,2.325581,0
422,1.40,834.042857,0.000000,0.000000,2,3.174603,0
423,2.55,765.825882,0.000000,0.000000,2,1.904762,0


In [22]:
xx.rename(columns={'폐사안락사 비율':'폐사안락사 비율'})

,면적,반려동물 가구밀도,폐사안락사 비율,보호소 밀도,동물병원약국개수,동물병원약국 밀도,target
0,1.34,1536.823881,11.194030,0.746269,11,8.208955,0
1,0.96,2583.450000,13.541667,1.041667,11,11.458333,0
2,1.13,2021.607080,16.814159,0.884956,10,8.849558,1
3,0.40,2389.237500,0.000000,0.000000,2,5.000000,0
4,2.52,1003.252381,0.000000,0.000000,14,5.555556,0
...,...,...,...,...,...,...,...
420,0.34,3238.011765,0.000000,0.000000,2,4.545455,0
421,0.26,2770.234615,0.000000,0.000000,2,2.325581,0
422,1.40,834.042857,0.000000,0.000000,2,3.174603,0
423,2.55,765.825882,0.000000,0.000000,2,1.904762,0


In [23]:
xx.to_csv('train.csv')